In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
metricsDir = './out/metrics'
graphsDir = './out/graphs'
df = pd.read_excel('results-javascript.xlsx')

In [12]:
def numericalMetrics(df,column,metricRows,metricsDir,graphsDir):
    mean = df[column].mean()
    median = df[column].median()
    stdev = df[column].std()
    stats = f'{column}\nMean: {mean}\nMedian: {median}\nStdDev: {stdev}'
    metricRows.append([column,mean,median,stdev])
    boxplot(df,column,graphsDir)

def boxplot(df,column,graphsDir):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    filter = (df[column] >= Q1 - 1.5 * IQR) & (df[column] <= Q3 + 1.5 *IQR)
    df_filtered = df.loc[filter]
    ax = df_filtered.boxplot(column,rot=10)
    ax.set_xlabel("")
    ax.set_ylabel(f"# of {column}")
    ax.set_title(column)
    # plt.show()
    fig = ax.get_figure()
    fig.savefig(f'./out/graphs/{column}.pdf')

def boolMetrics(df,column,boolRows,metricsDir,graphsDir):
    trueCount = df[column].sum()
    falseCount = (~df[column]).sum()
    total = df[column].shape[0]
    truePercentage = round((trueCount / total)*100,2)
    falsePercentage = round((falseCount / total)*100,2)
    boolRows.append([column,trueCount,falseCount,truePercentage,falsePercentage])

In [13]:
metricRows = []
boolRows = []
for col in df.columns:
    if df.dtypes[col] == 'int64':
        numericalMetrics(df,col,metricRows,metricsDir,graphsDir)
    elif df.dtypes[col] == 'bool':
        boolMetrics(df,col,boolRows,metricsDir,graphsDir)
metricsDF = pd.DataFrame(columns=["Metric","Mean","Median","StdDev"],data=metricRows)
boolDF = pd.DataFrame(columns=["Metric","True Count","False Count","True Percentage","False Percentage"],data=boolRows)
metricsDF.to_csv('./out/metrics/numericalMetrics.csv',index=False)
boolDF.to_csv('./out/metrics/boolMetrics.csv',index=False)

In [14]:
metricsDF

,Metric,Mean,Median,StdDev
0,commits,403.261891,107.0,1407.357441
1,branches,9.859782,3.0,39.171386
2,releases,17.779972,7.0,59.019702
3,contributors,11.350802,5.0,29.640616
4,watchers,20.072478,7.0,79.573109
5,stargazers,411.776143,50.0,2032.143878
6,forks,74.056509,14.0,469.134762
7,size,17389.014784,838.0,225132.826940
8,totalIssues,81.213143,15.0,382.875238
9,openIssues,16.113671,4.0,71.578183


In [15]:
boolDF

,Metric,True Count,False Count,True Percentage,False Percentage
0,isFork,367,57871,0.63,99.37
1,hasWiki,46399,11839,79.67,20.33
2,isArchived,4128,54110,7.09,92.91


In [16]:
metricColumns = ["commits","branches","releases","contributors","watchers","stargazers","forks","size","totalIssues","openIssues","totalPullRequests","openPullRequests"]
boolColumns = ["isFork","hasWiki","isArchived"]
boolCorrMatrix = df.corr().loc[metricColumns,boolColumns]
boolCorrMatrix.to_csv('./out/metrics/boolCorrMatrix.csv')
boolCorrMatrix

,isFork,hasWiki,isArchived
commits,0.056058,-0.052605,0.001425
branches,0.006534,-0.016180,0.000015
releases,-0.002622,-0.063137,0.015698
contributors,0.075003,-0.097156,0.014375
watchers,-0.009557,-0.051191,0.031544
stargazers,-0.009255,-0.061592,-0.014150
forks,-0.005603,-0.033382,-0.005349
size,0.007913,0.003170,-0.004973
totalIssues,-0.000889,-0.058219,0.000471
openIssues,0.001959,-0.029707,-0.005651


In [ ]:
metricColumns = ["commits","releases","contributors","forks","size","totalIssues","openIssues","totalPullRequests","openPullRequests"]
popularityMetrics = ["stargazers","forks","branches"]
popCorrMatrix = df.corr().loc[metricColumns,popularityMetrics]
popCorrMatrix.to_csv('./out/metrics/popCorrMatrix.csv')
popCorrMatrix